# Ejercicio SQL

Responderemos a una serie de preguntas sobre el dataset (San Francisco Fire Calls)[https://www.databricks.com/notebooks/gallery/SanFranciscoFireCallsAnalysis.html] usando SQL

## Carguemos los datos

El dataset que vamos a utilizar es (San Francisco Fire Calls)[https://www.databricks.com/notebooks/gallery/SanFranciscoFireCallsAnalysis.html]

Esta disponible en Databricks en este path /databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv

In [0]:
%fs ls /databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv

Cargamos los datos en un Dataframe usando el objeto (DataFrameReader)[https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/io.html] que nos proporciona la sesión de spark

In [0]:
spark.read

In [0]:
df_sf_fire_calls = (spark.read
                    .format("csv")
                    .option("header", "true")
                    .option("inferSchema", "true")
                    .load("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"))

# df_sf_fire_calls = (spark.read
#                     .csv("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv")
#                     .option("header", "true")
#                     .option("inferSchema", "true"))

df_sf_fire_calls.printSchema()

df_sf_fire_calls.show()

display(df_sf_fire_calls)

Este ejercicio queremos usar Spark SQL y trabajar con nuestros datos de una manera muy similar a como lo haríamos con una base de datos

Para ello carguemos el Dataframe en una tabla temporal sobre la que podremos ejecutar nuestras queries SQL

In [0]:
df_sf_fire_calls.createGlobalTempView("fire_calls")

In [0]:
%sql
select * from global_temp.fire_calls

### 1. Cuántos tipos distintos de llamadas ha recibido el departamento de incendios?

In [0]:
%sql
select count(distinct(CallType)) from global_temp.fire_calls

### 2. Cuales son los distintos tipos de llamadas?

In [0]:
%sql
select count(distinct(CallType)) from global_temp.fire_calls

### 3. Encuentra todas las llamadas con un retraso (delay) superior a 5 minutos

In [0]:
%sql
select * from global_temp.fire_calls where Delay > 5

### 3. Cuales son los tipos de llamada más frecuentes?

In [0]:
%sql
select CallType, count(*) as count from global_temp.fire_calls group by CallType

### 4. Que códigos postales (zipcode) observamos para los tipos de llamada más habituales


In [0]:
%sql
select CallType, count(*), collect_set(`Zipcode of Incident`) as count 
from global_temp.fire_calls 
group by CallType 
order by count desc 
limit 5

In [0]:
%sql
select CallType, `Zipcode of Incident`, count(*) as count 
from global_temp.fire_calls 
group by CallType, `Zipcode of Incident` 
order by count desc 
limit 5

### 5. Que barrios de San Francisco estan en los codigos postales 94102 y 94103?

In [0]:
%sql
select Neighborhood, `Zipcode of Incident` from global_temp.fire_calls where `Zipcode of Incident` = 94102 or `Zipcode of Incident` = 94103

In [0]:
%sql
select Neighborhood, `Zipcode of Incident` 
from global_temp.fire_calls 
where `Zipcode of Incident` = 94102 or `Zipcode of Incident` = 94103 
group by Neighborhood, `Zipcode of Incident`

## 6. Cuanto es la suma de los tiempso de respuesta de todas las llamadas las llamadas, la media, el minimo y el maximo

In [0]:
%sql
select avg(Delay), min(Delay), max(Delay)
from global_temp.fire_calls 

## 7. Cuanto años distintos de datos hay en el dataset (weekofyear, todate)

In [0]:
%sql
select distinct(year(`Call Date`))
from global_temp.fire_calls

## 8. En que semana del año 2018 hubo más llamadas que vecindarios tienen los peores tiempos de respuesta

In [0]:
%sql
select Neighborhood, weekofyear(`Call Date`), Delay, year(`Call Date`) 
from global_temp.fire_calls
where year(`Call Date`) = 2018